! pip install pyupbit
가상 화폐를 거래, 조회 등을 할 수 있는 업비트라는 회사(홈페이지)에서 테이터를 조회하는 라이브러리

In [ ]:
import pyupbit
# 가상 화폐의 데이터를 조회하기 위한 라이브러리
import sqlite3
# SQLite라는 데이터 베이스 프로그램을 실행하기 위한 라이브러리
import datetime
# 시간에 관련된 행동을 하기 위한 라이브러리
import pandas as pd
# 데이터를 시각화 하기 위해 불러오는 라이브러리

In [ ]:
coin_lists= pyupbit.get_tickers(fiat= 'KRW')
# 업비트에서 지원하는 암호 화폐 목록 중 특정시장(fiat)에서 매매 가능한 목록
print(coin_lists)

price_now= pyupbit.get_current_price(['KRW-BTC', 'KRW-ETH'])
# '한화-비트코인'과 '한화-이더리움'의 현재 가격을 불러오기
print(price_now)

가상 화폐의 데이터를 DB에 저장하고 생성  

In [ ]:
ticker= 'KRW-BTC'
# '한화-비트코인'의 데이터를 불러오기 위해.
interval= 'minutel'
# 분 단위로 변하는 주가를 간격으로 지정.
to= '2022-11-03 17:00'
# 지정한 시간 이전의 데이터를 불러오기 위해.
count= 200
# 불러올 데이터의 개수.
price_now= pyupbit.get_ohlcv(ticker= ticker, interval= interval, to= to, count= count)

db_path= r"C:\Users\user\Desktop\가상화폐_DB\coin.db"
# 생성할 DB의 경로

con= sqlite3.connect(db_path, isolation_level= None)
# 'connect' DB를 생성하고 연결하기 위한 함수, 
# 'isolation_level= None' update, insert, delete 등과 같이 DB의 데이터에 
# 변화를 주는 쿼리문에 대해 오토커밋을 해주는 함수

price_now.to_sql('BTC', con, if_exists= 'append')
# DB에 테이블이 존재할 때 '추가'라는 동작을 지정 

con.close

DB의 데이터 출력

In [ ]:
db_path= r"C:\Users\user\Desktop\가상화폐_DB\coin.db"

con= sqlite3.connect(db_path, isolation_level= None)
readed_df= pd.read_sql("SELECT * FROM 'BTC'", con, index_col= 'index')
# 'index_col=' 특정 columns를 지정
print(readed_df)

오늘 하루 동안 가상 화폐의 데이터 DB에 수집(2022-11-03) 

In [ ]:
def date_range(start, end):
    start= datetime.datetime.strptime(start, "%Y-%m-%d")
    # 'datetime.strptime' 날짜와 시간 형식의 문자열을 datetime으로 변환
    start= start + datetime.timedelta(days= 1)
    # 날짜를 비교하기 위한 함수
    end= datetime.datetime.strptime(end, "%Y-%m-%d")
    end= end + datetime.timedelta(days= 1)
    dates= [(start + datetime.timedelta(days= i)).strftime("%Y-%m-%d") for i
            in range((end-start).days+1)] # 'strftime' 날짜와 시간을 문자열로
    return dates

dates= date_range("2022-11-01","2022-11-02")

print(dates)

for day in reversed(dates):
    myDay= day + ' 00:00' # Ex)day= 2018-11-21 09:
    print(myDay)
    ticker= 'KRW-BTC'
    interval= 'minutel'
    to= myDay
    count= 1440
    price_now= pyupbit.get_ohlcv(ticker= ticker, interval= interval, to= to, count= count)

    print(price_now)
    
    db_path= r"C:\Users\user\Desktop\가상화폐_DB\coin.db"

    con= sqlite3.connect(db_path, isolation_level= None)
    price_now.to_sql('BTC', con, if_exists= 'append')
    
    con.close

중복된 값 제외

In [ ]:
db_path= r"C:\Users\user\Desktop\가상화폐_DB\coin.db"
con= sqlite3.connect(db_path, isolation_level= None)

readed_df= pd.read_sql("SELECT DISTINCT * FROM 'BTC'", con, index_col= 'index')
# 중복된 자료 제거
readed_df.to_sql('BTC_NEW', con, if_exists= 'replace')
# 'replace'= 테이블이 존재하면 기존 테이블 삭제 후 생성
print(readed_df)